# Forecasting Data

This is the **last** notebook of the series.

> Make sure you have executed all the previous ones. This notebook is entirely dependent upon the existance of a `model` directory, which is create upon the execution of the 3_Train.ipynb notebook.
---

# Objective

At last, once trained, our model should be able to perform some predictions. This notebook should forecast some cases and perform analysis on the results obtained.

---

# Loading the Trained Model

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras

df = pd.read_pickle('prep.pkl')
model = keras.models.load_model('model')
train_X = np.load('trainX.npy')
train_Y = np.load('trainY.npy')
test_X = np.load('testX.npy')
test_Y = np.load('testY.npy')
epochs = 100
model.summary()

2021-07-08 18:28:20.319217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-08 18:28:20.319233: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-07-08 18:28:21.155255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-08 18:28:21.155271: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-08 18:28:21.155282: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (HM01): /proc/driver/nvidia/version does not exist
2021-07-08 18:28:21.155430: I tensorflow/core/platform/cpu_feature_guard

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 3)                 96        
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4         
Total params: 100
Trainable params: 100
Non-trainable params: 0
_________________________________________________________________


# Forecasting with inputs 

Now that the model was loaded with the trained sets, the forecasting can finally be executed and evaluated

In [2]:
forecast_train = model.predict(train_X)
forecast_test = model.predict(test_X)

2021-07-08 18:28:22.334110: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2021-07-08 18:28:22.350213: I tensorflow/core/platform/profile_utils/cpu_utils.cc:114] CPU Frequency: 2000800000 Hz


## Evaluation

Since the data was normalized for training, it is important to denormalize it, so that the forecasting could be evaluated from better perspective.

In order to do that, it is important to have access to the data that was used for normalization. Since it was pickled in the previous notebook, the next cell rebuilds the scaller from the same base

In [3]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
stack_month_df = scaler.fit_transform(df['speed'].to_frame().values)

Now that the scaler is loaded, the evaluation can be performed using the __Root Mean Squared Error__:

In [4]:
trained_forecast = scaler.inverse_transform(forecast_train)
trained_expected = scaler.inverse_transform([train_Y])

test_forecasted = scaler.inverse_transform(forecast_test)
test_expected = scaler.inverse_transform([test_Y])

rmse = np.sum(np.sqrt(np.abs((trained_expected - trained_forecast.T) / trained_expected) ** 2)) / epochs
print("Root Mean Squared Error: {:.4f}".format(rmse))

Root Mean Squared Error: 1.9770


## Visual Evaluation

> To be completed...